In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression


In [82]:
def LR_hypothesis(w, X):
    """
    Обчислює гіпотезу лінійної регресії.

    Параметри:
    w (numpy.ndarray): Вектор параметрів (розмірність: (n+1, 1), де n - кількість ознак з урахуванням константного члена).
    X (numpy.ndarray): Матриця ознак (розмірність: (m, n+1), де m - кількість прикладів, n - кількість ознак з урахуванням константного члена).

    Повертає:
    numpy.ndarray: Вектор прогнозованих значень (розмірність: (m, 1)).
    """
    return X.dot(w)

def compute_cost(w, X, y):
    """
    Обчислює функцію втрат (Mean Squared Error).

    Параметри:
    w (numpy.ndarray): Вектор параметрів (розмірність: (n+1, 1), де n - кількість ознак з урахуванням константного члена).
    X (numpy.ndarray): Матриця ознак (розмірність: (m, n+1), де m - кількість прикладів, n - кількість ознак з урахуванням константного члена).
    y (numpy.ndarray): Вектор справжніх значень (розмірність: (m, 1)).

    Повертає:
    float: Значення функції втрат (Mean Squared Error).
    """
    m = len(y)
    predictions = LR_hypothesis(w, X)
    cost = (1 / (2 * m)) * np.sum(np.square(predictions - y))
    return cost

def gradient_descent_step(w, X, y, learning_rate):
    """
    Виконує один крок градієнтного спуску для оновлення параметрів.

    Параметри:
    w (numpy.ndarray): Вектор параметрів (розмірність: (n+1, 1), де n - кількість ознак з урахуванням константного члена).
    X (numpy.ndarray): Матриця ознак (розмірність: (m, n+1), де m - кількість прикладів, n - кількість ознак з урахуванням константного члена).
    y (numpy.ndarray): Вектор справжніх значень (розмірність: (m, 1)).
    learning_rate (float): Крок навчання.

    Повертає:
    numpy.ndarray: Оновлений вектор параметрів.
    """
    m = len(y)
    predictions = LR_hypothesis(w, X)
    errors = predictions - y
    gradient = (1 / m) * X.T.dot(errors)
    w -= learning_rate * gradient
    return w


In [83]:
def gradient_descent(X, y, w, learning_rate, num_iterations):
    """
    Функція для багатокрокового градієнтного спуску в лінійній регресії.

    Параметри:
    X (numpy.ndarray): Матриця ознак, включаючи біас (розмірність: m x n).
    y (numpy.ndarray): Вектор відгуків (розмірність: m x 1).
    w (numpy.ndarray): Початкові коефіцієнти моделі, включаючи біас (розмірність: n x 1).
    learning_rate (float): Швидкість навчання алгоритму градієнтного спуску.
    num_iterations (int): Кількість кроків градієнтного спуску.

    Повертає:
    numpy.ndarray: Оновлені коефіцієнти моделі після кількість_кроків ітерацій.
    """
    for _ in range(num_iterations):
        w = gradient_descent_step(w, X, y, learning_rate)
    return w

In [84]:
houses = pd.read_csv("housing.csv")
X = houses[['area', 'bedrooms', 'bathrooms']].to_numpy()
Y = houses.price.to_numpy()

In [85]:
houses.describe()

,price,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.693578
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,3.000000


In [86]:
def normalize_features(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    
    std[std == 0] = 1
    
    normalized_X = (X - mean) / std
    return normalized_X, mean, std

In [87]:
X_n, mean, std = normalize_features(X)

In [88]:
n = X.shape[1]
W = np.random.randn(n)
learned_W = gradient_descent(X_n, Y, W, 0.01, 10000)
coeffs = learned_W / std # un-normalize
print (coeffs)

[3.78762754e+02 4.06820034e+05 1.38604950e+06]


In [89]:
analitical_W = np.linalg.pinv(X.T @ X) @ X.T @ Y
analitical_W

array([3.72448352e+02, 3.68974672e+05, 1.37031315e+06])

In [91]:
print(f"Коефіцієнти ознак, знайдені метододом градієнтного спуску(area, bedrooms, bathrooms): {coeffs}")
print(f"Коефіцієнти ознак, знайдені аналітичним методом (area, bedrooms, bathrooms): {analitical_W}")

Коефіцієнти ознак, знайдені метододом градієнтного спуску(area, bedrooms, bathrooms): [3.78762754e+02 4.06820034e+05 1.38604950e+06]
Коефіцієнти ознак, знайдені аналітичним методом (area, bedrooms, bathrooms): [3.72448352e+02 3.68974672e+05 1.37031315e+06]
